In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice

import requests
import json

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
path = '경로 지정'

### 주어진 데이터

`song_keyword_culst1.xlsx` : 노래마다 `기본 정보 + 요약된 노래 가사 + 그걸 텍스트 전처리한 결과 + af 클러스터링 결과` 다 통합한 상태
`af_전체_장르.csv` : 전체 af DB, 장르 정보 포함
`노래_전체_장르.csv` : 전체 노래 DB, 장르 정보 포함

- 

In [7]:
total_af = pd.read_csv(path + '/힙합 제거 후/af_전체_장르.csv',index_col=0)
total_af

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0eFMbKCRw8KByXyWBw8WO7,0.529,0.469,1,-6.967,0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,0.577,0.729,7,-7.113,1,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,0.629,0.585,9,-7.969,1,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,0.596,0.707,1,-6.825,1,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25434,1AJZ9OpSs6V1jzUqdkdWPf,0.657,0.569,2,-6.287,1,0.0542,0.2010,0.000013,0.1260,0.399,102.055,"['art pop', 'pop']"
25435,3Q5c8dpE2aYMx97WnwRFGj,0.524,0.141,9,-19.818,1,0.0319,0.9190,0.944000,0.0770,0.356,78.698,"['art pop', 'pop']"
25436,7aObrOB2jVZJ2vJ79lQGjU,0.589,0.412,11,-6.226,0,0.0363,0.7200,0.000000,0.1530,0.703,74.954,"['art pop', 'pop']"
25437,7MlsUrIhUPvjoLZhjTLfFo,0.551,0.550,10,-5.339,1,0.0286,0.7770,0.000000,0.2020,0.342,101.190,"['art pop', 'pop']"


In [8]:
total_af['id'].duplicated().sum()

5012

In [9]:
drop_index = total_af[total_af['id'].duplicated()].index
print(drop_index)
total_af.drop(drop_index,inplace=True)
total_af

Int64Index([13106, 13107, 13230, 13231, 13232, 13233, 13234, 13235, 13236,
            13237,
            ...
            25379, 25380, 25381, 25382, 25383, 25384, 25385, 25386, 25387,
            25388],
           dtype='int64', length=5012)


,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0eFMbKCRw8KByXyWBw8WO7,0.529,0.469,1,-6.967,0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,0.577,0.729,7,-7.113,1,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,0.629,0.585,9,-7.969,1,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,0.596,0.707,1,-6.825,1,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25434,1AJZ9OpSs6V1jzUqdkdWPf,0.657,0.569,2,-6.287,1,0.0542,0.2010,0.000013,0.1260,0.399,102.055,"['art pop', 'pop']"
25435,3Q5c8dpE2aYMx97WnwRFGj,0.524,0.141,9,-19.818,1,0.0319,0.9190,0.944000,0.0770,0.356,78.698,"['art pop', 'pop']"
25436,7aObrOB2jVZJ2vJ79lQGjU,0.589,0.412,11,-6.226,0,0.0363,0.7200,0.000000,0.1530,0.703,74.954,"['art pop', 'pop']"
25437,7MlsUrIhUPvjoLZhjTLfFo,0.551,0.550,10,-5.339,1,0.0286,0.7770,0.000000,0.2020,0.342,101.190,"['art pop', 'pop']"


In [10]:
total_af_genre = total_af[['id','genre']]
total_af_genre

,id,genre
0,0eFMbKCRw8KByXyWBw8WO7,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,['japanese teen pop']
...,...,...
25434,1AJZ9OpSs6V1jzUqdkdWPf,"['art pop', 'pop']"
25435,3Q5c8dpE2aYMx97WnwRFGj,"['art pop', 'pop']"
25436,7aObrOB2jVZJ2vJ79lQGjU,"['art pop', 'pop']"
25437,7MlsUrIhUPvjoLZhjTLfFo,"['art pop', 'pop']"


In [11]:
total_af['id'].duplicated().sum()

0

# 본격적으로 분류 시작

In [14]:
one = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst1.xlsx", index_col=0)
two = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst2.xlsx", index_col=0)
thr = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst3.xlsx", index_col=0)
fou = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst4.xlsx", index_col=0)
fiv = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst5.xlsx", index_col=0)
six = pd.read_excel(path + "/노래 데이터 통합/song_keyword_culst6.xlsx", index_col=0)

one.head(1)

,id,url,Artist,PC1,PC2,PC3,clust,name,lyrics,lyrics_english,summary,key_word
0,7i5pytF8KaId51lIzatwdg,https://open.spotify.com/track/7i5pytF8KaId51l...,Fujii Kaze,0.755436,-0.31284,0.373133,0,Kiri Ga Naikara,キリがないから\nここで終わらすだけなの\nさもなけりゃ 永遠に 永遠に 彷徨うザマ\nあと...,Because there is no drill\nJust end here\nZama...,<pad> because there is no drill <unk> Because ...,drill unk drill unk drill end zama wanders for...


In [15]:
one.shape[0] + two.shape[0] + thr.shape[0] + fou.shape[0] + fiv.shape[0] + six.shape[0]

16766

In [17]:
song_data = pd.read_csv(path + '/힙합 제거 후/노래_전체_장르.csv',index_col=0)
song_data.head()

,id,url,Artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop']


In [18]:
song_data.iloc[0]

id                                             0eFMbKCRw8KByXyWBw8WO7
url                 https://open.spotify.com/track/0eFMbKCRw8KByXy...
Artist                                                      Jung Kook
danceability                                                    0.529
energy                                                          0.469
key                                                               1.0
loudness                                                       -6.967
mode                                                              0.0
speechiness                                                    0.0358
acousticness                                                   0.0899
instrumentalness                                             0.000015
liveness                                                        0.831
valence                                                         0.344
tempo                                                          87.864
genre               

In [19]:
clust = pd.concat([one[['id','Artist','clust','name']],two[['id','Artist','clust','name']],thr[['id','Artist','clust','name']],fou[['id','Artist','clust','name']],fiv[['id','Artist','clust','name']],six[['id','Artist','clust','name']]])
clust.head()

,id,Artist,clust,name
0,7i5pytF8KaId51lIzatwdg,Fujii Kaze,0,Kiri Ga Naikara
1,7r9v0CLTunOwlbvdOFFgZo,Fujii Kaze,0,Tsumi No Kaori
2,74Zrps4gc3qTF49oKpm6zF,Fujii Kaze,0,Cho Si Noccha Te
3,0o9zmvc5f3EFApU52PPIyW,Fujii Kaze,0,Shinunoga E-Wa
4,1KixkQVDUHggZMU9dUobgm,Ava Max,0,My Head & My Heart


In [25]:
song_clust = pd.merge(song_data, clust, how='inner',on='id')
print(song_clust.shape)
song_clust.head()

(16766, 18)


,id,url,Artist_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,Artist_y,clust,name
0,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop'],Jung Kook,4,Still With You
1,7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']",Powfu,5,death bed (coffee for your head)
2,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop'],Fujii Kaze,3,Nan-Nan
3,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop'],Fujii Kaze,3,Mo-Eh-Wa
4,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop'],Fujii Kaze,1,YASASHISA


In [22]:
def song_mood(x):
    if x == 0:
        return [9,7, 2,6]#['enthusiasm','love','fun','happiness'] #[9,7, 2,6]
    elif x == 1:
        return [8,12]#['hate','anger'] #[8,12]
    elif x == 2:
        return [1,0,6]#['empty','sadness','love'] #[1,0,6]
    elif x == 3:
        return [11]#['relief'] #[11]
    elif x == 4:
        return [4]#['worry'] #[4]
    else:
        return [10]#['boredom'] #[10]

In [26]:
song_clust['sentiment'] =song_clust['clust'].apply(song_mood)
print(song_clust.shape)
song_clust.head()

(16766, 19)


,id,url,Artist_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,Artist_y,clust,name,sentiment
0,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864,['k-pop'],Jung Kook,4,Still With You,[4]
1,7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,"['sad lo-fi', 'sad rap']",Powfu,5,death bed (coffee for your head),[10]
2,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065,['japanese teen pop'],Fujii Kaze,3,Nan-Nan,[11]
3,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973,['japanese teen pop'],Fujii Kaze,3,Mo-Eh-Wa,[11]
4,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858,['japanese teen pop'],Fujii Kaze,1,YASASHISA,"[8, 12]"


In [28]:
song_clust.to_excel('song_clust.xlsx')

In [18]:
song_clust['sentiment'].values

array([list([4]), list([10]), list([11]), ..., list([10]),
       list([1, 0, 6]), list([1, 0, 6])], dtype=object)

In [21]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(song_clust['sentiment'].values)
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [22]:
mlb.classes_

array([ 0,  1,  2,  4,  6,  7,  8,  9, 10, 11, 12])

In [23]:
columns = ['empty','sadness','enthusiasm','worry','love','fun','hate','happiness','boredom','relief','anger']

In [24]:
new_data = pd.DataFrame(columns = columns,data=labels)
new_data

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
16761,0,0,0,0,0,0,0,0,1,0,0
16762,0,0,0,0,0,0,1,0,0,0,1
16763,0,0,0,0,0,0,0,0,1,0,0
16764,1,1,0,0,1,0,0,0,0,0,0


In [25]:
audio_feature = song_clust.iloc[:,3:-5]

In [26]:
audio_feature

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864
1,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026
2,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065
3,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973
4,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858
...,...,...,...,...,...,...,...,...,...,...,...
16761,0.572,0.231,0.0,-6.316,0.0,0.0334,0.9530,0.000004,0.1110,0.332,96.732
16762,0.563,0.473,0.0,-4.147,1.0,0.0361,0.1770,0.000002,0.1080,0.342,69.926
16763,0.589,0.412,11.0,-6.226,0.0,0.0363,0.7200,0.000000,0.1530,0.703,74.954
16764,0.551,0.550,10.0,-5.339,1.0,0.0286,0.7770,0.000000,0.2020,0.342,101.190


In [27]:
song_sentiment = pd.concat([audio_feature,new_data],axis=1)

In [28]:
song_sentiment.shape

(16766, 22)

In [29]:
song_sentiment

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,...,0,0,1,0,0,0,0,0,0,0
1,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,...,0,0,0,0,0,0,0,1,0,0
2,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,...,0,0,0,0,0,0,0,0,1,0
3,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,...,0,0,0,0,0,0,0,0,1,0
4,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16761,0.572,0.231,0.0,-6.316,0.0,0.0334,0.9530,0.000004,0.1110,0.332,...,0,0,0,0,0,0,0,1,0,0
16762,0.563,0.473,0.0,-4.147,1.0,0.0361,0.1770,0.000002,0.1080,0.342,...,0,0,0,0,0,1,0,0,0,1
16763,0.589,0.412,11.0,-6.226,0.0,0.0363,0.7200,0.000000,0.1530,0.703,...,0,0,0,0,0,0,0,1,0,0
16764,0.551,0.550,10.0,-5.339,1.0,0.0286,0.7770,0.000000,0.2020,0.342,...,1,0,0,1,0,0,0,0,0,0


# Training

In [ ]:
song_sentiment.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger'],
      dtype='object')

In [64]:
X = song_sentiment[['danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
Y= song_sentiment[['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger']]

In [65]:
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.529,0.469,1.0,-6.967,0.0,0.0358,0.0899,0.000015,0.8310,0.344,87.864
1,0.726,0.431,8.0,-8.765,0.0,0.1350,0.7310,0.000000,0.6960,0.348,144.026
2,0.577,0.729,7.0,-7.113,1.0,0.2210,0.3940,0.000156,0.1440,0.692,180.065
3,0.629,0.585,9.0,-7.969,1.0,0.0572,0.4300,0.000000,0.3570,0.296,81.973
4,0.596,0.707,1.0,-6.825,1.0,0.0490,0.1110,0.000000,0.0926,0.262,133.858
...,...,...,...,...,...,...,...,...,...,...,...
16761,0.572,0.231,0.0,-6.316,0.0,0.0334,0.9530,0.000004,0.1110,0.332,96.732
16762,0.563,0.473,0.0,-4.147,1.0,0.0361,0.1770,0.000002,0.1080,0.342,69.926
16763,0.589,0.412,11.0,-6.226,0.0,0.0363,0.7200,0.000000,0.1530,0.703,74.954
16764,0.551,0.550,10.0,-5.339,1.0,0.0286,0.7770,0.000000,0.2020,0.342,101.190


In [66]:
Y

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
16761,0,0,0,0,0,0,0,0,1,0,0
16762,0,0,0,0,0,0,1,0,0,0,1
16763,0,0,0,0,0,0,0,0,1,0,0
16764,1,1,0,0,1,0,0,0,0,0,0


In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

In [68]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(11736, 11)
(11736, 11)
(5030, 11)
(5030, 11)


## Model 학습

In [72]:
import torch

In [73]:
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [74]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [75]:
train_dataset = TensorDataset(torch.tensor(X_train.values, dtype=torch.float32).to(device),
                              torch.tensor(Y_train.values, dtype=torch.float32).to(device))
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [ ]:
class MyModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.model = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Linear(in_features, 800),
            nn.ReLU(),

            nn.BatchNorm1d(800),            # nn.Dropout(0.1),
            nn.Linear(800, 600),
            nn.ReLU(),

            nn.BatchNorm1d(600),            # nn.Dropout(0.1),
            nn.Linear(600, 400),
            nn.ReLU(),

            nn.BatchNorm1d(400),
            nn.Linear(400, out_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
X_train.shape[1], Y_train.shape[1]

(11, 11)

In [ ]:
train_dataloader

In [ ]:
model = MyModel(X_train.shape[1], Y_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 0.05, 10)
criterion = nn.BCELoss()
max_epoch = 20

In [ ]:
for epoch in range(max_epoch):
    model.train()
    for i, (x_batch, y_batch) in enumerate(train_dataloader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        preds = model(x_batch)

        optimizer.zero_grad()

        #print(preds.shape)

        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch: {epoch}, train loss: {loss.item():12.5f}')

    model.eval()
    with torch.no_grad():
        train, y_train = train_dataset.tensors
        train, y_train = train.to(device), y_train.to(device)
        train_preds = model(train)
        train_loss = criterion(train_preds, y_train).item()

Epoch: 0, train loss:      0.71957
Epoch: 1, train loss:      0.08527
Epoch: 2, train loss:      0.04599
Epoch: 3, train loss:      0.07978
Epoch: 4, train loss:      0.02731
Epoch: 5, train loss:      0.01892
Epoch: 6, train loss:      0.01183
Epoch: 7, train loss:      0.01464
Epoch: 8, train loss:      0.02013
Epoch: 9, train loss:      0.01909
Epoch: 10, train loss:      0.00848
Epoch: 11, train loss:      0.03922
Epoch: 12, train loss:      0.00693
Epoch: 13, train loss:      0.00827
Epoch: 14, train loss:      0.04224
Epoch: 15, train loss:      0.03796
Epoch: 16, train loss:      0.01018
Epoch: 17, train loss:      0.01283
Epoch: 18, train loss:      0.01089
Epoch: 19, train loss:      0.00666


# TEST

In [ ]:
model.eval()
with torch.no_grad():
    probs = model(torch.tensor(X_test.values, dtype=torch.float32).to(device))

In [ ]:
probs = probs.cpu().numpy()

In [ ]:
probs.shape

(5030, 11)

In [ ]:
X_test.index

Int64Index([ 7375, 13980,  6924,  3666,  1272,  1888,   337, 13736, 13469,
             3229,
            ...
            11298, 13406, 13546,  3198,  4217,  5927, 14435,  5018, 12243,
             1168],
           dtype='int64', length=5030)

In [ ]:
Y_train.columns

Index(['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger'],
      dtype='object')

In [ ]:
df = pd.DataFrame(probs,index=X_test.index,columns = Y_train.columns)
df

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
7375,0.000715,0.000585,0.000043,0.504970,0.000607,0.000046,0.000071,0.000045,0.429658,0.000054,0.000042
13980,0.000338,0.000349,0.000206,0.000456,0.000081,0.000220,0.000146,0.000186,0.000828,1.000000,0.000140
6924,0.000600,0.000477,0.998632,0.005271,0.997470,0.998747,0.000580,0.998626,0.000072,0.000314,0.000526
3666,0.001874,0.001564,0.000341,0.004507,0.003601,0.000343,0.000139,0.000387,0.999715,0.000160,0.000090
1272,0.000402,0.000422,1.000000,0.000174,1.000000,1.000000,0.000187,1.000000,0.000724,0.000040,0.000128
...,...,...,...,...,...,...,...,...,...,...,...
5927,0.015377,0.014470,0.000161,0.000429,0.027162,0.000162,0.994201,0.000163,0.000178,0.000104,0.994521
14435,0.000156,0.000151,0.000106,0.000485,0.000074,0.000106,0.999917,0.000107,0.000255,0.001757,0.999925
5018,0.970921,0.971406,0.000230,0.001418,0.943607,0.000269,0.050129,0.000246,0.000246,0.000144,0.051796
12243,0.000596,0.000627,1.000000,0.001501,0.999988,1.000000,0.000333,1.000000,0.001066,0.000253,0.000327


# 모델 저장

In [ ]:
import joblib

joblib.dump(model, 'CustomModel.pkl')

['CustomModel.pkl']

# 모델 돌리기

In [70]:
import joblib

In [76]:
model = joblib.load('CustomModel.pkl')
model

AttributeError: Can't get attribute 'MyModel' on <module '__main__'>

In [ ]:
model.eval()
with torch.no_grad():
    probs = model(torch.tensor(X.values, dtype=torch.float32).to(device))

In [ ]:
probs = probs.cpu().numpy()

In [ ]:
probs

array([[2.91179545e-04, 4.02083679e-04, 7.23257079e-04, ...,
        2.25911426e-04, 2.98665018e-05, 2.05380056e-05],
       [4.65553530e-05, 5.30805919e-05, 2.71695247e-03, ...,
        9.99995351e-01, 1.13003654e-04, 1.05436041e-03],
       [6.87650230e-04, 9.59205325e-04, 3.23531480e-04, ...,
        3.42301064e-04, 9.99999523e-01, 2.54359300e-04],
       ...,
       [1.60487965e-04, 1.36235074e-04, 7.81684145e-02, ...,
        9.30233955e-01, 2.33624494e-04, 5.72232879e-04],
       [9.99715149e-01, 9.99727666e-01, 1.67227583e-04, ...,
        5.48835378e-04, 4.59687784e-03, 4.37649054e-04],
       [1.00000000e+00, 1.00000000e+00, 3.54219919e-05, ...,
        3.29769886e-04, 3.51998460e-05, 4.64355340e-03]], dtype=float32)

In [77]:
df = pd.DataFrame(probs,index=X.index,columns = Y.columns)
df

NameError: name 'probs' is not defined

In [ ]:
df.shape

(16766, 11)

In [ ]:
df.to_excel('CustomModel_sentiment.xlsx')

In [78]:
CustomModel_sentiment = pd.read_excel('CustomModel_sentiment.xlsx',index_col=0)

In [79]:
CustomModel_sentiment

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,2.911795e-04,4.020837e-04,0.000723,1.000000,0.000268,0.000786,0.000016,0.000905,0.000226,0.000030,0.000021
1,4.655535e-05,5.308059e-05,0.002717,0.000437,0.000715,0.002430,0.001243,0.002587,0.999995,0.000113,0.001054
2,6.876502e-04,9.592053e-04,0.000324,0.000625,0.000448,0.000291,0.000311,0.000333,0.000342,1.000000,0.000254
3,1.522953e-03,2.080173e-03,0.000245,0.000927,0.001087,0.000204,0.000420,0.000249,0.000339,0.999903,0.000401
4,9.784889e-04,9.201588e-04,0.000283,0.000440,0.000199,0.000349,1.000000,0.000242,0.000417,0.000897,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
16761,4.753783e-07,6.276099e-07,0.000051,0.000090,0.000007,0.000048,0.000067,0.000055,1.000000,0.000002,0.000077
16762,7.284960e-04,6.270141e-04,0.000191,0.000621,0.000095,0.000227,1.000000,0.000144,0.000347,0.000716,1.000000
16763,1.604880e-04,1.362351e-04,0.078168,0.000073,0.061114,0.070167,0.000758,0.068545,0.930234,0.000234,0.000572
16764,9.997151e-01,9.997277e-01,0.000167,0.000388,0.998012,0.000142,0.000415,0.000186,0.000549,0.004597,0.000438


In [82]:
song_final_data = pd.concat([song_clust['id'],song_clust['name'],song_clust['Artist_x'],CustomModel_sentiment,song_clust['genre']],axis=1)
song_final_data.columns = ['id', 'name', 'Artist', 'empty', 'sadness', 'enthusiasm', 'worry',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger',
       'genre']

In [83]:
song_final_data

,id,name,Artist,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger,genre
0,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,2.911795e-04,4.020837e-04,0.000723,1.000000,0.000268,0.000786,0.000016,0.000905,0.000226,0.000030,0.000021,['k-pop']
1,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,4.655535e-05,5.308059e-05,0.002717,0.000437,0.000715,0.002430,0.001243,0.002587,0.999995,0.000113,0.001054,"['sad lo-fi', 'sad rap']"
2,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,6.876502e-04,9.592053e-04,0.000324,0.000625,0.000448,0.000291,0.000311,0.000333,0.000342,1.000000,0.000254,['japanese teen pop']
3,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,1.522953e-03,2.080173e-03,0.000245,0.000927,0.001087,0.000204,0.000420,0.000249,0.000339,0.999903,0.000401,['japanese teen pop']
4,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,9.784889e-04,9.201588e-04,0.000283,0.000440,0.000199,0.000349,1.000000,0.000242,0.000417,0.000897,1.000000,['japanese teen pop']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16761,5JL1HLtR9jrlQraBCT1DL9,We Don't Have To Take Our Clothes Off,Various Artists,4.753783e-07,6.276099e-07,0.000051,0.000090,0.000007,0.000048,0.000067,0.000055,1.000000,0.000002,0.000077,"['art pop', 'pop']"
16762,4RyYlpna1veWNpLvGDyqnE,Us,Various Artists,7.284960e-04,6.270141e-04,0.000191,0.000621,0.000095,0.000227,1.000000,0.000144,0.000347,0.000716,1.000000,"['art pop', 'pop']"
16763,7aObrOB2jVZJ2vJ79lQGjU,L.O.V.E.,Various Artists,1.604880e-04,1.362351e-04,0.078168,0.000073,0.061114,0.070167,0.000758,0.068545,0.930234,0.000234,0.000572,"['art pop', 'pop']"
16764,7MlsUrIhUPvjoLZhjTLfFo,Until I Found You - Em Beihold Version,Various Artists,9.997151e-01,9.997277e-01,0.000167,0.000388,0.998012,0.000142,0.000415,0.000186,0.000549,0.004597,0.000438,"['art pop', 'pop']"


In [ ]:
song_final_data.to_excel('song_final_data.xlsx')